In [1]:
from mpi4py import MPI
from dolfinx import mesh, io, fem
from petsc4py.PETSc import ScalarType
import numpy as np
import ufl

In [2]:
# simulation parameters
MESH_CORNERS=((0.0, 0.0), (1.0, 1.0))
N = (32, 32)
TIME_STEP = 0.01
FINAL_TIME = 0.05
PRESSURE_CORRECTION_TOLERANCE = 1e-3
DEBUG = True

In [3]:
domain = mesh.create_rectangle(comm=MPI.COMM_WORLD, points=MESH_CORNERS, n=N) 

#with io.XDMFFile(MPI.COMM_WORLD, "mesh.xdmf", "w") as xdmf:
#    xdmf.write_mesh(domain)

# physical parameters
rho = fem.Constant(domain, ScalarType(1.0)) # fluid density
mu = fem.Constant(domain, ScalarType(0.005)) # fluid viscosity

from math import pi, tan
smallest_angle = pi/4 # get smaller angle in mesh with help of external software
k_mu = ScalarType(18*mu/tan(smallest_angle)) # SI penalty, asumed constant viscosity and polynomials of deegre 2
# see https://doi.org/10.1016/j.cam.2006.08.029 eq. (48)

In [4]:
velocity_function_space = fem.functionspace(domain, ("Discontinuous Lagrange", 2, (2,))) # Lagrange deegre 2, vector 2x1
pressure_function_space = fem.functionspace(domain, ("Discontinuous Lagrange", 1)) # Lagrange deegre 2, scalar

u = ufl.TrialFunction(velocity_function_space)
p = ufl.TrialFunction(pressure_function_space)

v = ufl.TestFunction(velocity_function_space)
q = ufl.TestFunction(pressure_function_space)

u_n = fem.Function(velocity_function_space) # one time step before (known)
u_n_1 = fem.Function(velocity_function_space) # two time step before (known)
# initialized as zero, so initial condition is u(x, 0) = 0

decay_factor =  np.exp(-2.0 * np.pi * np.pi * mu.value * TIME_STEP)
u_n_1.interpolate(lambda x: np.vstack((-np.sin(np.pi * x[1]) * np.cos(np.pi * x[0]),
                                          np.sin(np.pi * x[0]) * np.cos(np.pi * x[1]))))
u_n.interpolate(lambda x: np.vstack((-np.sin(np.pi * x[1]) * np.cos(np.pi * x[0]) * decay_factor,
                                          np.sin(np.pi * x[0]) * np.cos(np.pi * x[1]) * decay_factor)))


w = fem.Function(velocity_function_space, name="w") 
w.x.array[:] = 2 * u_n.x.array - u_n_1.x.array

In [5]:
from ufl import FacetNormal, dx, ds, dS, dot, outer, div, jump, avg, grad, conditional, ge, inner, transpose
from dolfinx.fem.petsc import create_matrix, create_vector

# second order backward difference for time
# first time step uses first order backward differences 
diff_coef_1 = fem.Constant(domain, 1.0)
diff_coef_2 = fem.Constant(domain, -1.0)
diff_coef_3 = fem.Constant(domain, 0.0)

n = FacetNormal(domain)
u_upwind = conditional(ge(dot(w('+'), n('+')), 0), u('+'), u('-')) # get upwind direction 

# define forms and allocate matrices
m_form = (rho/TIME_STEP) * diff_coef_1 * dot(u, v) * dx
m_compiled = fem.form(m_form)
M = create_matrix(m_compiled)

# w('+') no estaba en el paper, lo inclui porque si no daba error
# PUEDE QUE OUTER SEA AL REVÉS
r_form = - dot(u, div(rho * outer(v, w))) * dx \
         + dot(w, n * dot(u, rho*v)) * ds \
         + dot(w('+'), n('+') * dot(u_upwind, jump(rho*v))) * dS \
         + mu * inner(grad(u) + transpose(grad(u)), grad(v)) * dx \
         + k_mu * dot(jump(u), jump(v)) * dS \
         - dot(dot(mu*(grad(u) + transpose(grad(u))), n), v) * ds \
         - dot(dot(avg(mu*(grad(u) + transpose(grad(u)))), n('+')), jump(v)) * dS \
         - dot(dot(mu*(grad(v) + transpose(grad(v))), n), u) * ds \
         - dot(dot(avg(mu*(grad(v) + transpose(grad(v)))), n('+')), jump(u)) * dS 
r_compiled = fem.form(r_form)
R = create_matrix(r_compiled)

b_form = - p * div(v) * dx \
         + avg(p) * dot(n('+'), jump(v)) * dS \
         + p * dot(n, v) * ds
b_compiled = fem.form(b_form)
B = create_matrix(b_compiled)

c_form = - dot(u, grad(q)) * dx \
         + dot(avg(u), n('+')) * jump(q) * dS \
         + dot(u, n) * q * ds 
c_compiled = fem.form(c_form)
C = create_matrix(c_compiled)

d_form = (rho/TIME_STEP) * dot(diff_coef_2*u_n + diff_coef_3*u_n_1, v) * dx
d_compiled = fem.form(d_form)
d = create_vector(d_compiled)

e_form = fem.Constant(domain, ScalarType(0.))*q*dx
e_compiled = fem.form(e_form)
e = create_vector(e_compiled)

In [6]:
# assemble the forms into matrices/vectors
from dolfinx.fem.petsc import assemble_matrix, assemble_vector
from time import perf_counter

M.zeroEntries()
start_M = perf_counter()
assemble_matrix(M, m_compiled)
M.assemble()
end_M = perf_counter()

R.zeroEntries()
start_R = perf_counter()
assemble_matrix(R, r_compiled)
R.assemble()
end_R = perf_counter()

B.zeroEntries()
start_B = perf_counter()
assemble_matrix(B, b_compiled)
B.assemble()
end_B = perf_counter()

C.zeroEntries()
start_C = perf_counter()
assemble_matrix(C, c_compiled)
C.assemble()
end_C = perf_counter()

d.zeroEntries()
start_d = perf_counter()
assemble_vector(d, d_compiled)
d.assemble()
end_d = perf_counter()

e.zeroEntries()
start_e = perf_counter()
assemble_vector(e, e_compiled)
e.assemble()
end_e = perf_counter()

In [7]:
from petsc4py import PETSc

# intitialize solver for aproximate velocity
solver_velocity = PETSc.KSP().create(domain.comm)
solver_velocity.setType(PETSc.KSP.Type.GMRES)
pc_velocity = solver_velocity.getPC()
pc_velocity.setType(PETSc.PC.Type.ASM) 
pc_velocity.setASMType(PETSc.PC.ASMType.BASIC)
solver_velocity.setTolerances(rtol=1e-15, atol=1e-15, max_it=100)

A = M.copy()
A.axpy(1.0, R, structure=PETSc.Mat.Structure.DIFFERENT_NONZERO_PATTERN)
solver_velocity.setOperators(A) 

# initialize solver for M inverse
solver_M = PETSc.KSP().create()
solver_M.setType('preonly')
solver_M.setOperators(M)
pc_M = solver_M.getPC()
pc_M.setType('lu')

# define PETSc matrix for P = C * M⁻¹ * B.
class PMat:
    def __init__(self, B, C, solver_M):
        self.B = B
        self.C = C
        self.solver_M = solver_M

    def mult(self, mat, p_in, p_out):
        # p_out = C*inv(M)*(B dot p_in)
        rows_b = B.getSize()[0] 
        tmp = PETSc.Vec().create(comm=PETSc.COMM_WORLD)
        tmp.setSizes(rows_b)
        tmp.setUp()
        self.B.mult(p_in, tmp) # tmp = B * p_in
        y = tmp.duplicate()
        self.solver_M.solve(tmp, y) # y = inv(M) *(B*p_in)
        self.C.mult(y, p_out) # p_out = C * y
        tmp.destroy()

#initialize P
size_rows, _ = C.getSizes()
_, size_cols = B.getSizes()
P = PETSc.Mat().createPython([size_rows, size_cols], context=PMat(B, C, solver_M))
P.setUp()

# initialize solver for pressure correction 
solver_pressure = PETSc.KSP().create()
solver_pressure.setType(PETSc.KSP.Type.GMRES)
pc_pressure = solver_pressure.getPC()
pc_pressure.setType(PETSc.PC.Type.HYPRE)
pc_pressure.setHYPREType("boomeramg")  
solver_pressure.setTolerances(rtol=1e-15, atol=1e-15, max_it=100)
solver_pressure.setOperators(P) 

In [ ]:
from dolfinx.fem.petsc import assemble_matrix, assemble_vector

p_write = fem.Function(pressure_function_space)

time = TIME_STEP # not 0, initial conditions are given
p_guess = PETSc.Vec().createMPI(B.getSizes()[1])
p_guess.set(0.0)  # initial guess for initial time: zero pressure

u_file = io.VTXWriter(domain.comm, "u.bp", u_n)
p_file = io.VTXWriter(domain.comm, "p.bp", p_write)
u_file.write(time)
p_file.write(time)

while time < FINAL_TIME:
    if DEBUG:
        i = 0
        start_iter = perf_counter()
    
    # iterative pressure correction
    while True:  
        # step 1: aproximate velocity u_star
        # RHS = d - B dot p_guess
        rhs_velocity = d.copy() 
        temp = d.duplicate() # size of u
        B.mult(p_guess, temp)
        rhs_velocity.axpy(-1.0, temp)
        
        # LHS = A = M + R
        A = M.copy()
        A.axpy(1.0, R, structure=PETSc.Mat.Structure.DIFFERENT_NONZERO_PATTERN)

        # solve for u_star
        u_star = d.duplicate()  # create vector with same size as d
        solver_velocity.setOperators(A) # !! SET OPERATOR NO VA ACA
        solver_velocity.solve(rhs_velocity, u_star)
        
        # step 2: pressure correction
        # compute y = inv(M) * (B dot p_guess) by solving M dot y = B dot p_guess
        rhs_y = temp.duplicate() # same size as u, since B maps from p to u   
        B.mult(p_guess, rhs_y)
        y = rhs_y.duplicate()
        solver_M.setOperators(M)
        solver_M.solve(rhs_y, y) 

        #   RHS = C*inv(M)*B dot p_guess - e + C dot u_star = C dot y - e + C dot u_star.
        rhs_pressure = e.copy() # size of p
        rhs_pressure.scale(-1.0) 
        temp = e.duplicate() # size of p
        C.mult(u_star, temp)
        rhs_pressure.axpy(1.0, temp) 
        C.mult(y, temp)  
        rhs_pressure.axpy(1.0, temp)

        # LHS = C*inv(M)*B = P
        # !!! LOS SET OPERATORS VAN UN CICLO MAS AFUERA, YA QUE ACA ADENTRO NO CAMBIAN
        solver_pressure.setOperators(P) 

        # solve for p_new
        p_new = p_guess.duplicate() 
        solver_pressure.solve(rhs_pressure, p_new)
        
        # step 3: velocity correction
        # compute u = - inv(M) * B dot (p_new-p_guess) by solving M dot u = - B dot (p_new-p_guess) 
        dp = p_new.copy()
        dp.axpy(-1.0, p_guess)  
        temp = d.duplicate() # size of u
        B.mult(dp, temp)  
        if i>0:
            u_old = u_new.copy()
        u_new = u_star.duplicate()
        solver_M.solve(temp, u_new)

        # update p_guess for next iteration 
        p_new.copy(result=p_guess)

        # convergence test2 
        if i>0:
            u_residue2 = u_new.copy()
            u_residue2.axpy(-1.0, u_old)
            print(u_residue2.norm())
        
        # check convergence
        u_residue = u_star.copy()
        u_residue.axpy(-1.0, u_new)
        if u_residue.norm() <= PRESSURE_CORRECTION_TOLERANCE:
            if DEBUG:
                print(f"Time step: {time/TIME_STEP}, finished pressure correction iterations.")
            break

        if DEBUG:
            i += 1
            if i%1==0:
                #print(f"Time step: {time/TIME_STEP}, pressure iteration {i}, residue: {u_residue.norm()}.")
                temp = start_iter
                start_iter = perf_counter()
                #print(f"Average pressure iteration runtime (seconds): {(start_iter - temp)/10}")
            #if i==25: # debug
             #   raise KeyboardInterrupt

            

    
    # re-assign u_n, u_n_1 and w considering new solution
    u_n_1.x.array[:] = u_n.x.array
    u_n.x.array[:] = u_new.getArray()
    w.x.array[:] = 2 * u_n.x.array - u_n_1.x.array

    # save solution to xdmf file
    p_write.x.array[:] = p_new.getArray()
    u_file.write(time)
    p_file.write(time)
    
    # re-assemble time dependent forms
    R.zeroEntries()
    assemble_matrix(R, r_compiled)
    R.assemble()

    d.zeroEntries()
    assemble_vector(d, d_compiled)
    d.assemble()
    
    e.zeroEntries()
    assemble_vector(e, e_compiled)
    e.assemble()
    
    time += TIME_STEP
    if time > TIME_STEP:
        diff_coef_1.value = 3/2
        diff_coef_2.value = -2.0
        diff_coef_3.value = 1/2

        # re-assemble M since diff_coef_1 has changed
        M.zeroEntries()
        assemble_matrix(M, m_compiled)
        M.assemble()

u_file.close()
p_file.close()

6.0661248584629825e+28
6.83750597986425e+42
7.893291216278013e+56
8.816992589712787e+70
9.143296231252783e+84
1.144924988134375e+99
1.1932702175413087e+113
1.2716094758026763e+127
1.3681763026589489e+141
1.3040319943414757e+141
1.8348394844503516e+140
5.875652717043663e+139
6.341242840053537e+139
2.1950537646765753e+140
1.6191526231108318e+140
1.4073658723310937e+140
3.41959453985601e+139
7.356395613289696e+139
9.17372046741311e+139
2.4434699228981344e+140
4.5692731918895647e+139


In [ ]:
x = np.array([1, 2, 3])
np.exp(-4.0*pi*pi*mu.value*time)